In [1]:
# https://github.com/AlaaSedeeq/Convolutional-Autoencoder-PyTorch/blob/main/Model/Autoencoder.py
import json
from pathlib import Path
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import rasterio as rio
from rasterio.plot import reshape_as_image
import torch
import torch.nn as nn   
import torch.optim as optim
import torch.nn.functional as F


from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, RandomCrop


from utils import *

warnings.filterwarnings("ignore", category=RuntimeWarning)
import numpy as np
from tqdm import tqdm 
from collections import namedtuple
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from utils import prepare_data, decode_label, show_examples, Swish, weight_init, compare_results

# Local imports
from mlp import ImageDataset, MLP, train_test_ae_mlp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Constants
BATCH_SIZE = 1028
DATA_DIR = Path(r"C:\Users\Peter\gh\rasmussen-705.603\data\FinalProject")
EPOCHS = 30
GAMMA = 0.1
LAT_SPACE_SIZES = [2, 4, 16, 64, 256,]
LRS = [0.5, 0.1]
OUTPUT_BANDS = ["blue", "green", "red", "nir08", "swir16", "swir22", "ndvi", "qa"]
STEP_SIZES = [5, 10, 15]
TR_VAL_SPLIT = 0.9
WEIGHT_DECAY = 5e-4

In [2]:
# Define directories
raw_dir = DATA_DIR / "raw"
interim_dir = DATA_DIR / "interim"
processed_dir = DATA_DIR / "processed"
cogs_dir = interim_dir / "_cogs-4-30"
autoencoder_dir = processed_dir / "autoencoder"
models_dir = interim_dir / "models"
scores_dir = interim_dir / "scores"
models_dir.mkdir(exist_ok=True, parents=True)
scores_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# encoded = model.encoder(batch)
# encoded = encoded.view(-1, model.CNN_flatten)
# latent_space1 = model.latent_space1(encoded)
# latent_space2 = model.latent_space2(latent_space1)
# latent_space3 = model.latent_space3(latent_space2)
# latent_space3.shape, batch.shape
# decoded = latent_space3.view(-1, model.CNN[1], model.CNN[2], model.CNN[3])


In [3]:
# Create PyTorch datasets and loaders
dataset = ImageDataset(cogs_dir, OUTPUT_BANDS, slice_frac=0.01)

tr_val_size = int(TR_VAL_SPLIT * len(dataset))
te_size = len(dataset) - tr_val_size
tr_size = int(TR_VAL_SPLIT * tr_val_size)
val_size = tr_val_size - tr_size

tr_val_dataset, te_dataset = torch.utils.data.random_split(dataset, [tr_val_size, te_size])
tr_dataset, val_dataset = torch.utils.data.random_split(tr_val_dataset, [tr_size, val_size])


tr_loader = DataLoader(tr_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
te_loader = DataLoader(te_dataset, batch_size=BATCH_SIZE, shuffle=False)

len(tr_loader), len(val_loader), len(te_loader)

(2504, 279, 310)

In [4]:

epochs = EPOCHS


tr_val_losses = []
te_losses = []
for step_size in STEP_SIZES:
    for lr in LRS:

        print(f"Train model with step_size={step_size} and lr={lr}.")
        model = MLP(
            in_features=8,
            hidden_neurons=2,
            )
        model.to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=WEIGHT_DECAY, fused=True)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=GAMMA)
        tr_val_losses_, te_losses_ = train_test_ae_mlp(
            model,
            tr_loader, val_loader, te_loader,
            criterion, optimizer, scheduler,
            lr, step_size, epochs,
            models_dir, scores_dir,
            device,
        )
        tr_val_losses.extend(tr_val_losses_)
        te_losses.extend(te_losses_)


tr_val_summary = pd.DataFrame(tr_val_losses)
te_summary = pd.DataFrame(te_losses)

tr_val_summary.to_csv(scores_dir / "tr_val_summary.csv")
te_summary.to_csv(scores_dir / "te_summary.csv")

Train model with step_size=5 and lr=0.5.


RuntimeError: No active exception to reraise

In [144]:
optimal_step_size = int(tr_val_summary.sort_values(by="val_loss").iloc[0].to_dict()["step_size"])
optimal_lr = tr_val_summary.sort_values(by="val_loss").iloc[0].to_dict()["lr"]
optimal_step_size, optimal_lr

(10, 0.5)

In [147]:
model_src = models_dir / f"mlp-autoencoder-{optimal_step_size}-{optimal_lr:.3f}.pt"


True

In [148]:
new_model = AE(in_features=8, hidden_neurons=2)
new_model.load_state_dict(torch.load(model_src))


<All keys matched successfully>